In [ ]:
import pywt
from enum import Enum
import cv2
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt

## Функции преобразования текста и вспомогательные классы/

In [ ]:
class ColorSpaceType(Enum):
    RGB = 'RGB'
    YCrCb = 'YCrCb'

class SubbandType(Enum):
    LL = "LL"
    LH = "LH"
    HL = "HL"
    HH = "HH"

def text_to_bits(text: str) -> str:
    # кодируем строку в UTF-8
    data = text.encode("utf-8")
    # каждый байт превращаем в 8 бит
    return ''.join(f"{byte:08b}" for byte in data)

def bits_to_text(bits: str) -> str:
    # берём только полные байты
    bits = bits[: (len(bits) // 8) * 8]

    # переводим 8-битные блоки в байты
    bytes_arr = bytearray(
        int(bits[i:i+8], 2)
        for i in range(0, len(bits), 8)
    )

    # декодируем как UTF-8
    return bytes_arr.decode("utf-8", errors="replace")

def get_image_channel_data(image_path: str, image_color_space: ColorSpaceType):
    if image_color_space is ColorSpaceType.RGB:
        img = Image.open(image_path).convert('RGB')
        img_array = np.array(img)

        return img_array
    
    elif image_color_space is ColorSpaceType.YCrCb:
        img = Image.open(image_path).convert('RGB')
        img_array = np.array(img)
        img_ycbcr = cv2.cvtColor(img_array, cv2.COLOR_RGB2YCrCb)

        return img_ycbcr
    
    else:
        raise ValueError('Invalid color space.')
    
def save_image_channel_data(image_data: np.ndarray, save_image_path: str, image_color_space: ColorSpaceType):
    if image_color_space is ColorSpaceType.RGB:
        Image.fromarray(image_data).save(save_image_path)
        return
    
    elif image_color_space is ColorSpaceType.YCrCb:
        cbcr_image_data = cv2.cvtColor(image_data, cv2.COLOR_YCrCb2RGB)
        Image.fromarray(cbcr_image_data).save(save_image_path)
        return
    
    else:
        raise ValueError('Invalid color space.')

def loss_percentage_chars(original: str, extracted: str) -> float:
    """
    Возвращает процент символов, которые отличаются между original и extracted
    """
    # берём длину меньшей строки
    length = max(len(original), len(extracted))
    if length == 0:
        return 0.0
    
    # считаем несовпадающие позиции
    mismatches = 0
    for i in range(length):
        c1 = original[i] if i < len(original) else None
        c2 = extracted[i] if i < len(extracted) else None
        if c1 != c2:
            mismatches += 1
    
    return 100.0 * mismatches / length

def compare_strings(str1, str2):
    """Сравнивает две строки и показывает различия"""
    if len(str1) != len(str2):
        print(f"Длины разные: {len(str1)} vs {len(str2)}")
    
    min_len = min(len(str1), len(str2))
    differences = []
    
    for i in range(min_len):
        if str1[i] != str2[i]:
            differences.append((i, str1[i], str2[i]))
    
    if differences:
        print("Найдены различия:")
        for pos, char1, char2 in differences:
            print(f"  Позиция {pos}: '{char1}' vs '{char2}'")
    else:
        print("Строки идентичны (в пределах общей длины)")
    
    return len(differences) == 0 and len(str1) == len(str2)

## Функция заполнения контейнера (массива байт)

In [ ]:
def fill_container(container: np.ndarray[np.float32], text: str, Q: int):
    bits = text_to_bits(text)
    
    if len(bits) > len(container):
        raise ValueError(f"Контейнер слишком мал для встраивания текста: необходимо {len(bits)}, есть {len(container)}")
    
    modified_container = container.copy()

    # Квантование: используем округление, чтобы симметрично работать с отриц. значениями
    for idx, bit in enumerate(bits):
        c = container[idx]
        q = int(np.round(c / Q))
        if bit == '0':
            if q % 2 == 1:
                q -= 1
        else:
            if q % 2 == 0:
                q += 1
        modified_container[idx] = q * Q

    return modified_container, len(bits)

## Функция получения текста из контейнера

In [ ]:
def extract_container(container: np.ndarray[np.float32], text_length: int, Q: int) -> str:
    data_bits = ''
    for i in range(0, text_length):
        c = container[i]
        q = int(np.round(c / Q))
        data_bits += '1' if (q % 2 == 1) else '0'

    return bits_to_text(data_bits)

## Основные функции встраивания текста в изображение и получение теста из изображения

In [ ]:
def embed_text(image_path: str, out_path: str, text: str, color_space: ColorSpaceType = ColorSpaceType.RGB, target_subband: str = 'LL', channel: int = 0, Q: int = 4) -> int:
    image_data = get_image_channel_data(image_path, color_space)
    channel_data = image_data[:, :, channel].astype(np.float32)

    # DWT (используем periodization для более стабильного поведения)
    LL, (LH, HL, HH) = pywt.dwt2(channel_data, 'haar', mode='periodization')
    coeffs_dict = {
        'LL': LL,
        'LH': LH,
        'HL': HL,
        'HH': HH
    }
    selected_subband: np.ndarray[np.float32] = coeffs_dict[target_subband]
    subband = selected_subband.flatten()
    #

    subband, bits_length = fill_container(subband, text, Q)

    # записываем обратно и делаем IDWT
    modied_subband = subband.reshape(selected_subband.shape)
    coeffs_dict[target_subband] = modied_subband
    modified_coeffs = (coeffs_dict['LL'], 
                           (coeffs_dict['LH'], 
                            coeffs_dict['HL'], 
                            coeffs_dict['HH']))
    modified_channel_data = pywt.idwt2(modified_coeffs, 'haar', mode='periodization')

    image_data[:, :, channel] = modified_channel_data
    save_image_channel_data(image_data, out_path, color_space)

    return bits_length

In [ ]:
def extract_text(image_path: str, text_length: int, color_space: ColorSpaceType = ColorSpaceType.RGB, target_subband: str = 'LL', channel: int = 0, Q: int = 4) -> str:
    image_data = get_image_channel_data(image_path, color_space)
    channel_data = image_data[:, :, channel].astype(np.float32)

    # DWT (используем periodization для более стабильного поведения)
    LL, (LH, HL, HH) = pywt.dwt2(channel_data, 'haar', mode='periodization')
    coeffs_dict = {
        'LL': LL,
        'LH': LH,
        'HL': HL,
        'HH': HH
    }
    selected_subband: np.ndarray[np.float32] = coeffs_dict[target_subband]
    subband = selected_subband.flatten()

    string = extract_container(subband, text_length, Q)

    return string

### Тестовые данные

In [ ]:
used_image = 'img/marsik-960x1280.png'
output_image = 'embedded/marsik-960x1280-embedded.png'

# used_image = 'img/marsik-960x1280.jpg'
# output_image = 'embedded/marsik-960x1280-embedded.jpg'

# used_image = 'img/marsik-600x800.png'
# output_image = 'embedded/marsik-600x800-embedded.png'

# used_image = 'img/marsik-300x400.png'
# output_image = 'embedded/marsik-300x400-embedded.png'

# used_image = 'img/marsik-150x200.png'
# output_image = 'embedded/marsik-150x200-embedded.png'

# used_image = 'img/cat.jpg'
# output_image = 'embedded/cat-embedded.jpg'

# used_image = 'img/forest.jpg'
# output_image = 'embedded/forest-embedded.jpg'

secret_string = "Silence in an era of noise: How to find yourself in a world that never stops Our world is immersed in a continuous, intrusive hum. This is not just the physical noise of megacities, but a fundamental information and social backdrop that has become our new habitat. We wake up to the vibration of our smartphones, scroll through our news feeds at breakfast, and are immersed in a whirlwind of notifications, messaging apps, and endless online meetings at work. In the evening, as we try to relax, we unconsciously scroll through social media, consuming the carefully curated lives of others. This permanent digital noise creates the illusion of hyper-connection, but paradoxically leads to deep loneliness and distraction. We know more about the world, but less about ourselves. We have thousands of \"friends,\" but sometimes we don't have anyone to share our true sadness or joy with."

### Запуск на тестовых данных

In [ ]:
Q = 6

bits_lenght = embed_text(used_image,
           output_image,
           secret_string,
           color_space=ColorSpaceType.YCrCb,
           target_subband='HL',
           channel=0,
           Q=Q)
result = extract_text(output_image,
                      text_length = bits_lenght,
                      color_space=ColorSpaceType.YCrCb,
                      target_subband='HL',
                      channel=0,
                      Q=Q)
print(f'Исходный текст    ({len(secret_string)}):\t{secret_string}')
print(f'Извлечённый текст ({len(result)}):\t{result}')
print(f'Процент потерь: {loss_percentage_chars(secret_string, result)}')